In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import torch # type: ignore
import sys; sys.path.append('../')
from  giagrad.tensor import Tensor
import numpy as np
from giagrad.display import draw_dot
import torch.nn as nn
import string
import math
from itertools import chain
from numpy.lib.stride_tricks import as_strided

# Conv 2D

## Helpers

In [3]:
import string
from math import floor
alphabet = [chr(i) for i in chain(range(33,127), range(161, 670))]
# Channels out, Channels in, kernel Height, kernel Width
def kernel(Cout, Cin, kH, kW, a=0, b=4):
    return np.random.randint(a, b, (Cout, Cin, kH, kW))

def dataBatched(N, Cin, Hin, Win):
    return np.array(alphabet[:N*Cin*Hin*Win], dtype=object).reshape((N, Cin, Hin, Win))

def dataUnBatched(Cin, Hin, Win):
    return np.array(alphabet[:Cin*Hin*Win], dtype=object).reshape((Cin, Hin, Win))

## Forward

Para la convolucion se necesita convertir con as_strided el array anterior para que tenga dimension:
    $$(N, C_{in}, H_{in}, W_{in})$$
Donde 
$$
H_{out} = \left\lfloor 
            \frac{H_{in} + 2 \times padding[0] - dilation[0] \times (kernel\_size[0] -1) - 1}
                 {stride[0]} + 1
           \right\rfloor \\
W_{out} = \left\lfloor 
            \frac{W_{in} + 2 \times padding[0] - dilation[0] \times (kernel\_size[1] -1) - 1}
                 {stride[1]} + 1 
           \right\rfloor \\
$$

In [5]:
N, Cin, Hin, Win = 3, 4, 7, 7
kH, kW = 3, 3
Cout = 5
padding = (0, 0)
dilation = (1, 1)
stride = (1, 1)

Hout = floor((Hin + 2 * padding[0] - dilation[0] * (kH - 1) - 1) / stride[0]  + 1)
Wout = floor((Win + 2 * padding[1] - dilation[1] * (kW - 1) - 1) / stride[1]  + 1)

print('Hout: ', Hout)
print('Wout: ', Wout)

# define output shape
out_shape = (N, Cout, Hout, Wout)
print('out_shape: ', out_shape)

Hout:  5
Wout:  5
out_shape:  (3, 5, 5, 5)


In [6]:
# DATA
data = dataBatched(N, Cin, Hin, Win)
# random KERNEL
k = kernel(Cout, Cin, kH, kW)

strides debe tener una dimension mas que out_shape, ya que cada matrix $H_{in} \times W_{out}$ se debe dividir en todas las posibles mulitplicaciones del kernel.    

tenemos:

    - [ -6 , -5 , -4 , -3 , -2 , -1 ] * itemsize in bytes
    
    posicion -1: dilation[1], horizontal dilation
    posicion -2: Win * dilation[0], vertical dilation
    posicion -3: Hin*Win, jump to next Channel
    posicion -4: stride[1], horizontal stride
    posicion -5: Win*stride[0], vertical stride
    posicion -6: Hin*Win*Cin, next observation in Batched data

y el shape del strided array tiene que ser:
    
    -  [ -6 , -5 , -4 , -3 , -2 , -1 ]
    
    posicion -1: k2
    posicion -2: k1
    posicion -3: Cin
    posicion -4: Wout
    posicion -5: Hout
    posicion -6: N

In [10]:
stridedShape = (
    N,
    Hout, 
    Wout, 
    Cin,
    kH, 
    kW
)

strides = np.array(
    [Hin*Win*Cin,
     Win*stride[0],
     stride[1],
     Hin*Win,
     Win*dilation[0],
     dilation[1]]) * data.itemsize

print('strides no byte: ', strides/data.itemsize)
print('strides in bytes: ', strides)
print('strided output shape: ', stridedShape)
print('kernel shape: ', k.shape)
print('data shape: ', data.shape)

strides no byte:  [196.   7.   1.  49.   7.   1.]
strides in bytes:  [1568   56    8  392   56    8]
strided output shape:  (3, 5, 5, 4, 3, 3)
kernel shape:  (5, 4, 3, 3)
data shape:  (3, 4, 7, 7)


In [12]:
# expand data to apply tensordot
strided_array = as_strided(data, shape=stridedShape, strides=strides)

In [14]:
convolved = np.swapaxes(np.tensordot(k, strided_array, axes=([1, 2, 3], [3, 4, 5])), 0, 1)
print('convolved shape: ', convolved.shape)
print('desired shape: ', out_shape)

convolved shape:  (3, 5, 5, 5)
desired shape:  (3, 5, 5, 5)


## Test with MyGrad